In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.utils import to_categorical

In [ ]:
# Define a function to load and preprocess images and annotations
def load_and_preprocess_image(img_path, ann_path):
    img_path = img_path.numpy().decode('utf-8')
    ann_path = ann_path.numpy().decode('utf-8')
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 64))
    ann = cv2.imread(ann_path, 0)
    ann = cv2.resize(ann, (128, 64), interpolation=cv2.INTER_NEAREST)
    
    ann = np.clip(ann, 0, 29)
    ann = to_categorical(ann, num_classes=30)
    
    return img.astype(np.float32) / 255.0, ann.astype(np.float32)

In [ ]:
# Define a function to create a TensorFlow dataset
def create_tf_dataset(image_files, annotation_files, batch_size):
    def parse_function(img_path, ann_path):
        img, ann = tf.py_function(
            load_and_preprocess_image, 
            [img_path, ann_path], 
            [tf.float32, tf.float32]
        )
        img.set_shape((64, 128, 3))
        ann.set_shape((64, 128, 30))
        return img, ann
    
    dataset = tf.data.Dataset.from_tensor_slices((image_files, annotation_files))
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
# Define the U-Net architecture
def unet(input_size=(64, 128, 3), n_filters=16, n_classes=30):
    inputs = Input(input_size)
    conv1 = Conv2D(n_filters, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(n_filters, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(n_filters*2, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(n_filters*2, 3, activation='relu', padding='same')(conv2)
    up1 = Conv2DTranspose(n_filters, 3, strides=(2, 2), padding='same')(conv2)
    merge1 = concatenate([conv1, up1], axis=3)
    conv3 = Conv2D(n_filters, 3, activation='relu', padding='same')(merge1)
    conv3 = Conv2D(n_filters, 3, activation='relu', padding='same')(conv3)
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(conv3)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
import os
import tensorflow as tf

# Function to match image files with annotation files
def get_matched_files(image_dir, annotation_dir, img_suffix='_leftImg8bit.png', ann_suffix='_gtFine_color.png'):
    image_files = {}
    annotation_files = {}
    for dirpath, _, filenames in os.walk(image_dir):
        for filename in filenames:
            if filename.endswith(img_suffix):
                base_name = filename.replace(img_suffix, '')
                full_path = os.path.join(dirpath, filename)
                image_files[base_name] = full_path
    for dirpath, _, filenames in os.walk(annotation_dir):
        for filename in filenames:
            if filename.endswith(ann_suffix):
                base_name = filename.replace(ann_suffix, '')
                full_path = os.path.join(dirpath, filename)
                annotation_files[base_name] = full_path
    matched_images = []
    matched_annotations = []
    for base_name, img_path in image_files.items():
        ann_path = annotation_files.get(base_name)
        if ann_path:
            matched_images.append(img_path)
            matched_annotations.append(ann_path)
        else:
            print(f"Warning: No matching annotation file for {img_path}")
    return matched_images, matched_annotations

# Function to create a tf.data.Dataset
def create_tf_dataset(image_files, annotation_files, batch_size, num_classes):
    def _parse_function(img_path, ann_path):
        img_path = tf.convert_to_tensor(img_path, dtype=tf.string)
        ann_path = tf.convert_to_tensor(ann_path, dtype=tf.string)

        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, [64, 128])

        ann = tf.io.read_file(ann_path)
        ann = tf.image.decode_png(ann, channels=1)  # Assuming annotations are single-channel
        ann = tf.image.resize(ann, [64, 128], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        ann = tf.squeeze(ann, axis=-1)  # Remove the channel dimension
        ann = tf.one_hot(ann, num_classes)  # One-hot encode the annotations

        return img, ann

    dataset = tf.data.Dataset.from_tensor_slices((image_files, annotation_files))
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

# Main setup for training
image_dir = 'H:\\debian\\Cityscapes\\leftImg8bit_trainvaltest\\train'
annotation_dir = 'H:\\debian\\Cityscapes\\gtFine_trainvaltest\\train'
image_files, annotation_files = get_matched_files(image_dir, annotation_dir)
num_classes = 30  # Number of classes in your dataset
train_dataset = create_tf_dataset(image_files, annotation_files, batch_size=2, num_classes=num_classes)

# Initialize and compile the model
model = unet(input_size=(64, 128, 3), n_filters=16, n_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, verbose=1)

# Save the model
model.save('cityScapes.keras')

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the file
model = load_model('cityScapes.keras')

# Assuming you have a validation set prepared similarly to the training dataset:
image_files_val, annotation_files_val = get_matched_files(
    image_dir='H:\debian\Cityscapes\leftImg8bit_trainvaltest/test',
    annotation_dir='H:\debian\Cityscapes\gtFine_trainvaltest/test',
    img_suffix='_leftImg8bit.png',
    ann_suffix='_gtFine_color.png'
)
validation_dataset = create_tf_dataset(image_files_val, annotation_files_val, batch_size=2)

# Load the model if not already in memory (skip if model is still in memory)
# model = tf.keras.models.load_model('cityScapes.keras')

# Evaluate the model on the validation dataset
results = model.evaluate(validation_dataset)
print("Validation Loss: {:.4f}, Validation Accuracy: {:.4%}".format(results[0], results[1]))

# Optionally, for more detailed evaluation, you can use model.predict() on the validation dataset and calculate additional metrics like precision, recall, and F1-score


In [ ]:
# Assuming 'model' is your trained model
save_path = 'cityScapes_model'

# Save the model
tf.saved_model.save(model, save_path)

print(f"Model saved to {save_path}")


In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Load the .keras model
model = tf.keras.models.load_model('cityScapes.keras')

# Clear previous session to free up memory
K.clear_session()

# Prepare a representative dataset
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(image_files).batch(1).take(100):
        img = tf.io.read_file(input_value[0])
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, [64, 128])
        img = tf.cast(img, tf.float32) / 255.0
        yield [tf.expand_dims(img, axis=0)]

# Convert and quantize the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the converted and quantized model to a .tflite file
tflite_model_path = 'cityScapes_model_quant.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Quantized TensorFlow Lite model saved to {tflite_model_path}")


INFO:tensorflow:Assets written to: C:\Users\maith\AppData\Local\Temp\tmpsjiirfmu\assets


INFO:tensorflow:Assets written to: C:\Users\maith\AppData\Local\Temp\tmpsjiirfmu\assets


: 